In [28]:

!pip install langchain-google-genai langchain-core langchain langgraph

  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached anyi

In [29]:
import torch
import soundfile as sf
import numpy as np
import librosa
import os
import torchaudio
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langgraph.graph import StateGraph, END
from difflib import SequenceMatcher
import re
# !pip install sentencepiece transformers protobuf blobfile

In [42]:
import torchaudio
from transformers import SeamlessM4Tv2ForSpeechToText
from transformers import SeamlessM4TTokenizer, SeamlessM4TFeatureExtractor

audio_model = SeamlessM4Tv2ForSpeechToText.from_pretrained("ai4bharat/indic-seamless").to("cpu")
audio_processor = SeamlessM4TFeatureExtractor.from_pretrained("ai4bharat/indic-seamless")
audio_tokenizer = SeamlessM4TTokenizer.from_pretrained("ai4bharat/indic-seamless")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.45it/s]


In [43]:
def transcribe_audio_seamless(audio_path: str, model,processor,tokenizer) -> str:
    import torchaudio

    audio, orig_freq = torchaudio.load(audio_path)
    audio = torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=16_000) # must be a 16 kHz waveform array
    audio_inputs = processor(audio, sampling_rate=16_000, return_tensors="pt").to("cpu")
    text_out = model.generate(**audio_inputs, tgt_lang="hin")[0].cpu().numpy().squeeze()
    transcript = tokenizer.decode(text_out, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    return transcript


print(transcribe_audio_seamless("introduction.wav", audio_model,audio_processor,audio_tokenizer))

काशिल कुमार मैं आईआईटी जम्मू से हूं मैंने कंप्यूटर विज्ञान और इंजीनियरिंग में स्नातक की डिग्री प्राप्त की है मैं सी डॉट पर लगभग दो साल से काम कर रहा हूं मैं कई परियोजनाओं का हिस्सा रहा हूं वर्तमान में मैं मशीन लर्निंग और एआई में काम कर रहा हूं मैं कुछ अपना बनाने की कोशिश भी कर रहा हूं


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-m")
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-m")

c:\Users\kaush\.conda\envs\ai4bharat-py311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kaush\.cache\huggingface\hub\models--sarvamai--sarvam-m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]Error while downloading from https://cdn-lfs-us-1.h

In [47]:


# -------- State Definition --------
class TranscriptState(TypedDict):
    transcript: str
    sanitized: bool
    final_response: str
    error: str | None


# -------- Node 1: Sanitization --------
def sanitise_transcript_node(state: TranscriptState) -> TranscriptState:
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro-latest",
            google_api_key=os.environ["GOOGLE_API_KEY"]
        )
        prompt = PromptTemplate(
            template=(
                "You are an expert transcript editor.\n"
                "Your tasks are:\n"
                "1. Translate non-English parts to English.\n"
                "2. Polish grammar/technical phrasing.\n"
                "3. Ensure coherence.\n\n"
                "Transcript:\n{transcript}"
            ),
            input_variables=["transcript"]
        )
        chain = LLMChain(llm=llm, prompt=prompt, output_parser=StrOutputParser())
        result = chain.invoke({"transcript": state["transcript"]})
        return {
            "transcript": result,
            "sanitized": True,
            "final_response": "",
            "error": None
        }
    except Exception as e:
        return {
            "transcript": state["transcript"],
            "sanitized": False,
            "final_response": "",
            "error": str(e)
        }

# -------- Node 2: Gemini Logic/Response --------
def gemini_logic_node(state: TranscriptState) -> TranscriptState:
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro-latest",
            google_api_key=os.environ["GOOGLE_API_KEY"]
        )
        prompt = PromptTemplate(
            template=(
                "You are an intelligent assistant. Respond to the user query below:\n\n"
                "{transcript}"
            ),
            input_variables=["transcript"]
        )
        chain = LLMChain(llm=llm, prompt=prompt, output_parser=StrOutputParser())
        result = chain.invoke({"transcript": state["transcript"]})
        return {
            **state,
            "final_response": result,
            "error": None
        }
    except Exception as e:
        return {
            **state,
            "final_response": "",
            "error": str(e)
        }

# -------- Workflow Graph --------
def create_audio_to_response_graph():
    workflow = StateGraph(TranscriptState)
    workflow.add_node("respond", gemini_logic_node)
    workflow.set_entry_point("respond")
    workflow.add_edge("respond", END)
    return workflow.compile()

# -------- Run the Full Chain --------
def run_audio_pipeline(audio_path: str):
    transcript = transcribe_audio_seamless(audio_path,audio_model,audio_processor,audio_tokenizer)
    print("📝 Raw Transcript:", transcript)

    app = create_audio_to_response_graph()
    initial_state = TranscriptState(
        transcript=transcript,
        sanitized=False,
        final_response="",
        error=None
    )
    final_state = app.invoke(initial_state)

    if final_state["error"]:
        print(f"❌ Error: {final_state['error']}")
        return transcript, None
    return final_state["transcript"], final_state["final_response"]

# --------- Example Usage ---------
if __name__ == "__main__":
    audio_file = "introduction.wav"
    clean_transcript, gemini_reply = run_audio_pipeline(audio_file)
    print("\n✅ Clean Transcript:", clean_transcript)
    print("🤖 Gemini Reply:", gemini_reply)


📝 Raw Transcript: काशिल कुमार मैं आईआईटी जम्मू से हूं मैंने कंप्यूटर विज्ञान और इंजीनियरिंग में स्नातक की डिग्री प्राप्त की है मैं सी डॉट पर लगभग दो साल से काम कर रहा हूं मैं कई परियोजनाओं का हिस्सा रहा हूं वर्तमान में मैं मशीन लर्निंग और एआई में काम कर रहा हूं मैं कुछ अपना बनाने की कोशिश भी कर रहा हूं


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/gener

❌ Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key:

In [34]:
# !pip install google-generativeai
import google.generativeai as genai
import os

# Set your Google API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# List all available models
models = genai.list_models()

# Print available models and their capabilities
for model in models:
    print(f"Model: {model.name}")
    print(f"  Description: {model.description}")
    print(f"  Input Token Limit: {model.input_token_limit}")
    print(f"  Output Token Limit: {model.output_token_limit}")
    print(f"  Supported Generation Methods: {model.supported_generation_methods}")
    print("-" * 50)


Model: models/embedding-gecko-001
  Description: Obtain a distributed representation of a text.
  Input Token Limit: 1024
  Output Token Limit: 1
  Supported Generation Methods: ['embedText', 'countTextTokens']
--------------------------------------------------
Model: models/gemini-1.0-pro-vision-latest
  Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
  Input Token Limit: 12288
  Output Token Limit: 4096
  Supported Generation Methods: ['generateContent', 'countTokens']
--------------------------------------------------
Model: models/gemini-pro-vision
  Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
  Input Token Limit: 12288
  Output Token Limit: 4096
  Supported Generation Methods: ['generateCon